<a href="https://colab.research.google.com/github/Tclack88/DS-Unit-4-Sprint-1-NLP/blob/master/module4-topic-modeling/LS_DS_414_Topic_Modeling_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [2]:
# !python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=6b8ac9e1532af7f92e348df087c9021e76f532d16726af5e32283b0d1aad9f84
  Stored in directory: /tmp/pip-ephem-wheel-cache-h6q7c13u/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [102]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.9MB/s 
     |████████████████████████████████| 552kB 45.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=d847bd330ec1e70f3f1ea6f8d20d4102d82b064934daf68401d5343445fbbc74
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32040 sha256=f9b289851def97e8588890fccb9636062fa3ecc869981ff58503e326176b2457
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

import spacy

## Loading the data

In [3]:
!wget https://github.com/Tclack88/DS-Unit-4-Sprint-1-NLP/blob/master/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip?raw=true
!mv Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip?raw=true Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip
!unzip Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip

--2019-12-06 00:57:38--  https://github.com/Tclack88/DS-Unit-4-Sprint-1-NLP/blob/master/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip?raw=true
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Tclack88/DS-Unit-4-Sprint-1-NLP/raw/master/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip [following]
--2019-12-06 00:57:38--  https://github.com/Tclack88/DS-Unit-4-Sprint-1-NLP/raw/master/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Tclack88/DS-Unit-4-Sprint-1-NLP/master/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip [following]
--

In [63]:
df = pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


## pre-processing
There's too much unnecessary information in the dataset, we only need the topic (name) and the tokens (which we will derive from reviews.text)

In [0]:
df = df[['name','reviews.text']]
df.columns = ['item','review']

In [52]:
df.head()

,item,review
0,AmazonBasics AAA Performance Alkaline Batterie...,I order 3 of them and one of the item is bad q...
1,AmazonBasics AAA Performance Alkaline Batterie...,Bulk is always the less expensive way to go fo...
2,AmazonBasics AAA Performance Alkaline Batterie...,Well they are not Duracell but for the price i...
3,AmazonBasics AAA Performance Alkaline Batterie...,Seem to work as well as name brand batteries a...
4,AmazonBasics AAA Performance Alkaline Batterie...,These batteries are very long lasting the pric...


In [65]:
STOPWORDS = STOPWORDS.union(['amazon','item','amazonbasics','order','buy','purchase','christmas'])

# # "simple" tokenizer using simple_preprocess from gensim
# def tokenize(review):
#   return [token for token in simple_preprocess(review) if token not in STOPWORDS]
# df['tokens'] = df.review.apply(tokenize)

#Attempt at a more complex model with spacy, but it didn't do much better and takes longer
nlp = spacy.load('en_core_web_lg')
def tokenize(df):
  tokens = []
  for doc in nlp.pipe(df.review):
    doc_tokens = [token.lemma_.lower() for token in doc if (token.text.lower() not in STOPWORDS) and (token.is_punct == False) and (token.pos_ in ['NOUN','ADJ'])]
    tokens.append(doc_tokens)
  df['tokens'] = tokens
  return df

tokenize(df)

,item,review,tokens
0,AmazonBasics AAA Performance Alkaline Batterie...,I order 3 of them and one of the item is bad q...,"[bad, quality, backup, spring, pcs, aluminum, ..."
1,AmazonBasics AAA Performance Alkaline Batterie...,Bulk is always the less expensive way to go fo...,"[expensive, way, product]"
2,AmazonBasics AAA Performance Alkaline Batterie...,Well they are not Duracell but for the price i...,"[price, happy]"
3,AmazonBasics AAA Performance Alkaline Batterie...,Seem to work as well as name brand batteries a...,"[brand, battery, well, price]"
4,AmazonBasics AAA Performance Alkaline Batterie...,These batteries are very long lasting the pric...,"[battery, price, great]"
...,...,...,...
28327,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",I got 2 of these for my 8 yr old twins. My 11 ...,"[old, twin, yr, old, well, perfect, way]"
28328,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",I bought this for my niece for a Christmas gif...,"[niece, year, old]"
28329,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...","Very nice for light internet browsing, keeping...","[nice, light, internet, browsing, email, video..."
28330,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",This Tablet does absolutely everything I want!...,"[tv, bill, beautiful, screen]"


In [92]:
STOPWORDS = STOPWORDS.union(['amazon','item','amazonbasics','order','buy','purchase','christmas'])

#"simple" tokenizer using simple_preprocess from gensim
# def tokenize(review):
#   return [token for token in simple_preprocess(review) if token not in STOPWORDS]
# df['tokens'] = df.review.apply(tokenize)


#Attempt at a more complex model with spacy, but it didn't do much better and takes longer
nlp = spacy.load('en_core_web_lg')
def tokenize(df):
  tokens = []
  for doc in nlp.pipe(df.review):
    doc_tokens = [token.lemma_.lower() for token in doc if (token.text.lower() not in STOPWORDS) and (token.is_punct == False) and (token.pos_ in ['NOUN','ADJ'])]
    tokens.append(doc_tokens)
  df['tokens'] = tokens
  return df

tokenize(df)


# nlp = spacy.load('en_core_web_lg')
# tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)
# def tokenize(df):
#   tokens = []
#   # tokens = [list(doc.noun_chunks) for doc in tokenizer.pipe(df.review)]
#   for row in df.review:
#     doc_tokens = list(nlp(row).noun_chunks)
#     # doc_tokens = [token.lemma_.lower() for token in doc if (token.text.lower() not in STOPWORDS) and (token.is_punct == False) and (token.pos_ == 'NOUN')]
#     tokens.append(doc_tokens)
#   df['tokens'] = tokens
#   return df

# tokenize(df)


,item,review,tokens
0,AmazonBasics AAA Performance Alkaline Batterie...,I order 3 of them and one of the item is bad q...,"[bad, quality, backup, spring, pcs, aluminum, ..."
1,AmazonBasics AAA Performance Alkaline Batterie...,Bulk is always the less expensive way to go fo...,"[expensive, way, product]"
2,AmazonBasics AAA Performance Alkaline Batterie...,Well they are not Duracell but for the price i...,"[price, happy]"
3,AmazonBasics AAA Performance Alkaline Batterie...,Seem to work as well as name brand batteries a...,"[brand, battery, well, price]"
4,AmazonBasics AAA Performance Alkaline Batterie...,These batteries are very long lasting the pric...,"[battery, price, great]"
...,...,...,...
28327,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",I got 2 of these for my 8 yr old twins. My 11 ...,"[old, twin, yr, old, well, perfect, way]"
28328,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",I bought this for my niece for a Christmas gif...,"[niece, year, old]"
28329,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...","Very nice for light internet browsing, keeping...","[nice, light, internet, browsing, email, video..."
28330,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",This Tablet does absolutely everything I want!...,"[tv, bill, beautiful, screen]"


In [0]:
id_word_dict = corpora.Dictionary(df.tokens)

In [94]:
len(id_word_dict)

5520

In [0]:
# remove words that apear in less than 10 documents and over 90% of all documents
id_word_dict.filter_extremes(no_below=10, no_above=.95)

In [96]:
len(id_word_dict)

1329

In [58]:
id_word_dict[0] # verify filled corpus

'backup'

In [17]:
id_word_dict.token2id['battery']

0

## Determine number of topics and create corpus

In [43]:
df.item.unique()

array(['AmazonBasics AAA Performance Alkaline Batteries (36 Count)',
       'AmazonBasics Nylon CD/DVD Binder (400 Capacity)',
       'Amazon Echo ‚Äì White', 'Amazon Echo Show - Black',
       'Echo Spot Pair Kit (Black)',
       'Fire TV Stick Streaming Media Player Pair Kit',
       'AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging May Vary',
       'AmazonBasics Ventilated Adjustable Laptop Stand',
       'AmazonBasics Backpack for Laptops up to 17-inches',
       'AmazonBasics 11.6-Inch Laptop Sleeve',
       'AmazonBasics 15.6-Inch Laptop and Tablet Bag',
       'AmazonBasics External Hard Drive Case',
       'Expanding Accordion File Folder Plastic Portable Document Organizer Letter Size',
       'Cat Litter Box Covered Tray Kitten Extra Large Enclosed Hooded Hidden Toilet',
       'Amazon 9W PowerFast Official OEM USB Charger and Power Adapter for Fire Tablets and Kindle eReaders',
       'Kindle PowerFast International Charging Kit (for accelerated chargin

Looking at someone unique topics, I can see the following:

Everything seems to be able to fit into these3 categories: 
- tech
- pets
- homegoods

it may be further broken down into:

- batteries
- office supplies
- cases 
- echo 
- firestick
- laptop
- pet_stuff 
- charger_and_cables
- kitchenware 
- tablet 
- e_readers

I will experiment with just the 3 and continue to break it down if need be

In [0]:
# Gather all tokens of dataframe into a corpus:
corpus = [id_word_dict.doc2bow(t) for t in df.tokens]

In [98]:
corpus[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [0]:
import warnings
with warnings.catch_warnings():
  warnings.filterwarnings('ignore',category=DeprecationWarning)
  lda = LdaMulticore(corpus=corpus,
                   id2word=id_word_dict,
                   random_state=88,
                   num_topics = 4,
                   passes = 10,
                   workers=2)

In [113]:
lda.print_topics()

[(0,
  '0.155*"battery" + 0.086*"good" + 0.054*"price" + 0.033*"brand" + 0.029*"great" + 0.022*"quality" + 0.021*"life" + 0.016*"time" + 0.015*"cheap" + 0.014*"long"'),
 (1,
  '0.051*"app" + 0.030*"easy" + 0.023*"device" + 0.015*"screen" + 0.015*"gift" + 0.015*"feature" + 0.014*"light" + 0.014*"size" + 0.014*"new" + 0.012*"reader"'),
 (2,
  '0.195*"great" + 0.062*"book" + 0.057*"price" + 0.056*"product" + 0.034*"daughter" + 0.024*"easy" + 0.018*"screen" + 0.016*"excellent" + 0.015*"music" + 0.015*"game"'),
 (3,
  '0.146*"tablet" + 0.051*"kid" + 0.043*"old" + 0.041*"year" + 0.031*"good" + 0.029*"game" + 0.025*"kindle" + 0.020*"easy" + 0.017*"time" + 0.017*"great"')]

In this iteration, it looks like my categories are:
- batteries
- A generic "good quality" product (hint though it's related to an ereader or book
- 2 variations of a tablet or ebook reader

A previous lda (without using spacy and only the preprocessing features in gensim) it actually did a little better:

In [48]:
lda.print_topics()

[(0,
  '0.101*"batteries" + 0.030*"long" + 0.029*"work" + 0.018*"brand" + 0.015*"battery" + 0.014*"good" + 0.014*"time" + 0.012*"use" + 0.012*"brands" + 0.012*"like"'),
 (1,
  '0.061*"tablet" + 0.031*"kids" + 0.029*"loves" + 0.028*"old" + 0.028*"bought" + 0.024*"love" + 0.023*"year" + 0.023*"games" + 0.022*"use" + 0.020*"great"'),
 (2,
  '0.199*"great" + 0.151*"price" + 0.056*"works" + 0.046*"good" + 0.041*"battery" + 0.028*"life" + 0.021*"product" + 0.017*"value" + 0.015*"beat" + 0.014*"long"'),
 (3,
  '0.124*"good" + 0.060*"product" + 0.042*"great" + 0.042*"best" + 0.037*"recommend" + 0.037*"quality" + 0.027*"excellent" + 0.027*"deal" + 0.014*"highly" + 0.014*"happy"'),
 (4,
  '0.032*"kindle" + 0.029*"tablet" + 0.024*"use" + 0.020*"screen" + 0.020*"easy" + 0.014*"like" + 0.014*"love" + 0.014*"books" + 0.013*"reading" + 0.013*"read"')]

The categories chosen seem to be:
- batteries
- tables
- good product 1
- good product 2
- e-readers

## "Dope" visualizations

In [103]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

/usr/local/lib/python3.6/dist-packages/_pytest/mark/structures.py:426: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [104]:
pyLDAvis.gensim.prepare(lda, corpus, id_word_dict)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.285893 -0.074283       1        1  30.311064
1      0.144999 -0.161648       2        1  28.872166
3      0.116768 -0.037674       3        1  24.447351
2      0.024126  0.273605       4        1  16.369413, topic_info=     Category         Freq      Term        Total  loglift  logprob
12    Default  8757.000000     great  8757.000000  30.0000  30.0000
2     Default  8958.000000   battery  8958.000000  29.0000  29.0000
1013  Default  7348.000000    tablet  7348.000000  28.0000  28.0000
6     Default  2365.000000   product  2365.000000  27.0000  27.0000
9     Default  5094.000000     price  5094.000000  26.0000  26.0000
...       ...          ...       ...          ...      ...      ...
1240   Topic4   170.633713  parental   444.302490   0.8528  -5.1903
267    Topic4   194.965714       old  2540.638916  -0.7576  -5.0570
286    Topic4   165.187988   picture   405.044067   0.9128  -5.2228
699    Topic4   162.703339     phone   386.199036   0.9453  -5.2379
94     Topic4   164.981339      long   997.671265   0.0102  -5.2240

[278 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
926       4  0.937140      7yr
52        1  0.992569       aa
57        1  0.993366      aaa
54        2  0.432706     able
54        3  0.481363     able
...     ...       ...      ...
470       4  0.090624    young
1238      2  0.020358  youtube
1238      3  0.040716  youtube
1238      4  0.956833  youtube
807       4  0.998772       yr

[444 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling